In [15]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 5
EPOCHS = 40
CATEGORY = "moving" # "moving" or "stationary"
MODEL_NAME = f"category:{CATEGORY}_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 0
TEST_SIZE = 0
LEAVE_ONE_OUT = False
SAVE_MODEL = True

### This code takes in the type of activity, ie "stationary" or "moving" and trains a model for just classifying which phsyical activity of that category the data is  

In [16]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split
import split_by_student
import time


STATIONARY_ACTIVITIES = [
    "sitting",
    "standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
]

STATIONARY_ACTIVITIES_COMBINED = [
    "sitting_or_standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
]

MOVING_ACTIVITIES = [
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
]

DATA_DIRECTORY = "./all_respeck"

activity_dict = {"moving": MOVING_ACTIVITIES,
                       "stationary": STATIONARY_ACTIVITIES
            }

POSSIBLE_ACTIVITIES = activity_dict[CATEGORY]

if CATEGORY == "stationary":
    POSSIBLE_OUTCOMES = STATIONARY_ACTIVITIES_COMBINED
else:
    POSSIBLE_OUTCOMES = POSSIBLE_ACTIVITIES

LABEL_TO_INDEX = {label: idx for idx, label in enumerate(POSSIBLE_OUTCOMES)}


In [17]:
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in csv_dictionary.keys():
        
        
        activity = key.split("&")[0]
        if activity in POSSIBLE_ACTIVITIES:

            # iterates through each csv file for the activity 
            for csv_file in csv_dictionary[key]:
                if csv_file in file_names:
                    if gyro:
                        sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
                    else:
                        sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

                    # iterate through each generated sequence
                    for sequence in sequences:
                        if activity == "standing" or activity == "sitting":
                            tagged_data.append(("sitting_or_standing", sequence))
                        else:
                            tagged_data.append((activity, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [18]:
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [19]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [20]:
def leave_one_out():
    students = split_by_student.get_list_of_stutents()
    test_accuracies = []
    for test_student in students:
        print("testing student: " + str(test_student))
        print("training students: " + str([student for student in students if student != test_student]))
        
        test_files, training_files = split_by_student.get_list_of_files(test_student)

        tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
        tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

        train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
        test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

        model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=([], [])) #batch_size, epochs
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        test_accuracies.append(test_accuracy)
        print("for student " + str(test_student) + " the accuracy is " + str(test_accuracy))
        print("average accuracy so far: " + str(sum(test_accuracies)/len(test_accuracies)))
        print("number of students tested so far: " + str(len(test_accuracies)))
        time.sleep(3)

        
    print("Accuracy for each student:")
    print(", ".join([f"{student}: {accuracy}" for student, accuracy in zip(students, test_accuracies)]))
    print("Average overall accuracy:", sum(test_accuracies)/len(test_accuracies))


In [21]:
if LEAVE_ONE_OUT:
    leave_one_out()
    exit()  

train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE)


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, POSSIBLE_OUTCOMES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


if TEST_SIZE >0:
        test_loss, test_accuracy = model.evaluate(test_data, test_labels)
        print (f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


print("labels by index:")
for label, index in LABEL_TO_INDEX.items():
    print(f"{index}: {label}")


# Save the trained model
if SAVE_MODEL:
    model.save(f"models/demo_models/moving_classifier.keras")

Train Set: s36, s86, s27, s51, s21, s79, s11, s66, s5, s67, s70, s33, s43, s45, s16, s55, s83, s87, s56, s13, s17, s48, s65, s38, s9, s8, s50, s32, s18, s3, s30, s7, s23, s29, s42, s82, s57, s60, s22, s91, s63, s40, s59, s75, s88, s72, s92, s64, s74, s95, s34, s102, s77, s44, s1, s61, s52, s97, s84, s98, s100, s96, s46, s12, s15, s39, s93, s80, s35, s71, s54
Dev Set: 
Test Set: 
there are 11254 tagged sequences in the dataset
there are 0 tagged sequences in the dataset
there are 0 tagged sequences in the dataset
Epoch 1/40
2251/2251 [==============================] - 6s 2ms/step - loss: 0.7959 - accuracy: 0.6904
Epoch 2/40
2251/2251 [==============================] - 6s 3ms/step - loss: 0.3141 - accuracy: 0.8916
Epoch 3/40
2251/2251 [==============================] - 5s 2ms/step - loss: 0.2184 - accuracy: 0.9232
Epoch 4/40
2251/2251 [==============================] - 5s 2ms/step - loss: 0.1799 - accuracy: 0.9389
Epoch 5/40
2251/2251 [==============================] - 5s 2ms/step - loss